### Char Prediction using LSTM

1. Download data of Alice in Wonderland or Dracula from https://www.gutenberg.org/browse/scores/top in plain text format
2. Create an char_to_int map which maps each character used in the novel to an integer. example {a: 3}
3. Read data from the text file and do the following:
    3.1 Create a sliding window in which it takes in first 100 characters as the input sequence and 101th character as the output sequence. (It slides over every character).
    For example: 
        "Avul Pakir Jainulabdeen Abdul Kalam better known as A.P.J. Abdul Kalam"
        You should slide from "A" to the 100th char and 101th char will be your output.
        Then you should start sliding from "v" to the 100th char and 101th char will be your output.
    The input and the output sequence should be converted to their integer representation using the char_to_int map.
    With this you basically have two arrays seqIn and seqOut with each element containing integer representation of 100 characters and 1 character respectively.
    seqIn = [[10........15], [5.....25]...] seqOut = [5, 2, 5]
4. Now reshape your seqIn as (NumberOfSamples, 100, 1) - So you basically get this [[[10]........[15]], [[5]..... [25]]...]
5. One hot encode your seqOut using np_utils.to_categorical

6. Now create a simple model with LSTM followed by a Dense layer.

7. Then, given a seed sentence predict the next character using the model created.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import cross_validation
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot    
import plotly.graph_objs as go
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.cross_validation import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/home/user/classification/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
text1 = """ CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, and what is the use of a book, thought Alice without pictures or
conversations?

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

*    *    *    *    *    *    * """

In [4]:
def ascii(text):
    text_list = []
    for i in text:
        text_list.append(ord(i))  
    return text_list    
    

In [5]:
text_list = ascii(text1)

In [6]:
def sliding_window(text,length):
    sqin = []
    sqout = []
    for i in range(len(text)):
        output = []
        long = len(text)
        if((long-i)<(length+1)):
            break
        sqin.append(text[i:i+(length)])
        sqout.append(text[i+(length)])
    return sqin,sqout
            

In [7]:
sqin1,sqout = sliding_window(text_list,100)
sqout

[116,
 104,
 101,
 10,
 98,
 97,
 110,
 107,
 44,
 32,
 97,
 110,
 100,
 32,
 111,
 102,
 32,
 104,
 97,
 118,
 105,
 110,
 103,
 32,
 110,
 111,
 116,
 104,
 105,
 110,
 103,
 32,
 116,
 111,
 32,
 100,
 111,
 58,
 32,
 111,
 110,
 99,
 101,
 32,
 111,
 114,
 32,
 116,
 119,
 105,
 99,
 101,
 32,
 115,
 104,
 101,
 32,
 104,
 97,
 100,
 32,
 112,
 101,
 101,
 112,
 101,
 100,
 32,
 105,
 110,
 116,
 111,
 32,
 116,
 104,
 101,
 10,
 98,
 111,
 111,
 107,
 32,
 104,
 101,
 114,
 32,
 115,
 105,
 115,
 116,
 101,
 114,
 32,
 119,
 97,
 115,
 32,
 114,
 101,
 97,
 100,
 105,
 110,
 103,
 44,
 32,
 98,
 117,
 116,
 32,
 105,
 116,
 32,
 104,
 97,
 100,
 32,
 110,
 111,
 32,
 112,
 105,
 99,
 116,
 117,
 114,
 101,
 115,
 32,
 111,
 114,
 32,
 99,
 111,
 110,
 118,
 101,
 114,
 115,
 97,
 116,
 105,
 111,
 110,
 115,
 32,
 105,
 110,
 10,
 105,
 116,
 44,
 32,
 97,
 110,
 100,
 32,
 119,
 104,
 97,
 116,
 32,
 105,
 115,
 32,
 116,
 104,
 101,
 32,
 117,
 115,
 101,
 32,
 111,
 102,
 32,
 

In [8]:
max(sqout)

121

In [9]:
def reshaping(sqin):
    p = []
    for k in range(len(sqin)):
        q = []
        for i in range(len(sqin[0])):
            #print([sqin[k][i]])
            q.append([sqin[k][i]])
        p.append(q)
    return p

In [10]:
sqin = reshaping(sqin1)


In [11]:
def dataframe(sqin,sqout):
    para = pd.DataFrame(
        {'sqin': sqin,
         'sqout': sqout
        })
    return para

In [12]:
para = dataframe(sqin,sqout) 
para

,sqin,sqout
0,"[[32], [67], [72], [65], [80], [84], [69], [82...",116
1,"[[67], [72], [65], [80], [84], [69], [82], [32...",104
2,"[[72], [65], [80], [84], [69], [82], [32], [73...",101
3,"[[65], [80], [84], [69], [82], [32], [73], [46...",10
4,"[[80], [84], [69], [82], [32], [73], [46], [32...",98
5,"[[84], [69], [82], [32], [73], [46], [32], [68...",97
6,"[[69], [82], [32], [73], [46], [32], [68], [11...",110
7,"[[82], [32], [73], [46], [32], [68], [111], [1...",107
8,"[[32], [73], [46], [32], [68], [111], [119], [...",44
9,"[[73], [46], [32], [68], [111], [119], [110], ...",32


In [13]:
x_train, x_test = train_test_split(para, train_size = 0.8)
y_train = pd.DataFrame(x_train['sqout'])
y_test = pd.DataFrame(x_test['sqout'])
x_train.drop(['sqout'],axis=1,inplace=True)
x_test.drop(['sqout'],axis=1,inplace=True)

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [14]:
y_train['sqout'].max()

121

In [15]:
from keras.utils import np_utils
np_utils.to_categorical?
ytrain = np_utils.to_categorical(y_train, num_classes=255)
ytest = np_utils.to_categorical(y_test, num_classes=255)

In [16]:
xtrain = np.array(x_train)
xtest = np.array(x_test)

In [17]:
def fun(xtrain):
    #xtrain = np.array(x_train)
    spd = []
    for i in range(len(xtrain)):
        spd.append(xtrain[i][0])
    spd =  np.array(spd)   
    return spd

In [18]:
xtrain = fun(xtrain)
xtest = fun(xtest)

In [21]:
ytrain.shape

(445, 255)

In [22]:
xtrain.shape[1:]

(100, 1)

In [51]:
model = Sequential()
#model.add(LSTM(128,activation = 'relu', input_shape=(2,3)))
model.add(LSTM(64, input_shape=(xtrain.shape[1],xtrain.shape[2])))
#model.add(Dropout(0.2))
#model.add(Dense(600, activation='relu', input_shape=(3,)))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(ytrain.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics = ['accuracy'])
model.fit(xtrain, ytrain, epochs=5, batch_size=1, verbose=2,validation_split = .2)

Train on 356 samples, validate on 89 samples
Epoch 1/5
21s - loss: 3.4591 - acc: 0.1882 - val_loss: 3.3885 - val_acc: 0.1573
Epoch 2/5
23s - loss: 3.1092 - acc: 0.2163 - val_loss: 3.3696 - val_acc: 0.1573
Epoch 3/5
25s - loss: 2.9988 - acc: 0.2163 - val_loss: 3.3887 - val_acc: 0.1573
Epoch 4/5
25s - loss: 2.9937 - acc: 0.2135 - val_loss: 3.4556 - val_acc: 0.1573
Epoch 5/5
24s - loss: 2.9448 - acc: 0.2163 - val_loss: 3.5628 - val_acc: 0.1573


### character prediction using model

In [20]:
predict = """The simplest way to use the Keras LSTM model to make predictions is to first start off with a seed sequence 
as input, generate the next character then update the seed sequence to add the generated character on the end and 
trim off the first character. This process is repeated for as long as we want to predict new characters 
(e.g. a sequence of 1,000 characters in length)."""

In [21]:
pred = ascii(predict)


In [22]:
pred_slide,pred2 = sliding_window(pred,100)


In [23]:
pred_reshape = reshaping(pred_slide)


In [24]:
ar = np.array(pred_reshape)
ar.shape

(277, 100, 1)

In [25]:
result = model.predict(ar, batch_size=1, verbose=2)

In [26]:
result.shape

(277, 255)

In [27]:
np.argmax(result[10])

32

In [29]:
for i in range(result.shape[0]):
    res = np.argmax(result[i])
    test_result = chr(res)
    print(test_result)

 
 
 
 
 
 
 
 
t
*
 
 
t
 
 
 
 
 
 
t
 
 
 
 
 
 
 
 
t
 
 
 
t
 
 
 
 
t
 
 
 
 
 
 
 
 
 
t
 
 
 
 
t
 
 
 
 
 
 
t
 
 
 
t
 
 
 
 
t
 
 
 
 
 
 
 
 
t
 
 
t
 
 
 
t
 
 
 
t
 
 
 
 
 
 
 
 
 
t
 
 
 
 
 
 
 
 
 
t
 
 
t
 
 
 
t
 
 
 
t
 
 
 
t
*
 
 
 
 
t
 
 
 
t
 
 
 
t
 
 
 
 
 
t
 
 
 
 
 
 
 
 
 
 
t
 
 
 
 
t
 
 
 
 
 
 
 
t
 
 
t
 
 
 
 
 
 
 
 
t
 
 
 
t
 
 
t
 
 
 
 
t
 
 
t
 
 
t
 
 
 
 
t
 
 
t
 
 
 
 
 
 
 
t
 
 
 
t
 
 
 
 
 
 
 
 
 
 
t
*
 
 
 
 
 
 
 
t
 
 
 
 
 
 
 
 
t
 
 
t
 
 
 
 
 
t
 
 
 
 
 
 
 
 
 
 
t
 
 
t
 
 
 
 
 
 
 
